# Production model: constraint-style LP with Python/Gurobi
## [Jon Lee](https://sites.google.com/site/jonleewebpage/home), 10/20/2020

Notes:
* This example is meant to show how to:

   * do constraint-style LP's (as opposed to matrix style), though the model we are setting up is $\max\{c'x ~:~ Ax\leq b,~ x\geq \mathbf{0}\}$.
   
   * extract and print: primal and dual solutions, primal and dual slacks, and sensitivity information 
   
   * pass constraint names to Gurobi and then retrieve constraints from Gurobi by these names
   
   Edit: I also extend the code to solve the dual of the Production Problem example, as described in Section 2.1.
   
References:

* Jon Lee, "A First Course in Linear Optimization", Fourth Edition (Version 4.05), Reex Press, 2013-22.

MIT License

Copyright (c) 2020 Jon Lee

Permission is hereby granted, free of charge, to any person obtaining a copy 
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

In [15]:
%reset -f
import numpy as np
import gurobipy as gp
from gurobipy import GRB

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

In [16]:
# Some toy data
m=3
n=2
M=list(range(0,m))
N=list(range(0,n))
A = np.array([ [8, 5], [8, 6], [8, 7] ])
b = np.array([32, 33, 35])
c = np.array([3 ,2])
s= np.random.rand(m)

In [17]:
model = gp.Model()
model.reset()
x = model.addMVar(n)
s= model.addMVar(m)
revenueobjective = model.setObjective(sum(c[j]*x[j] for j in N), GRB.MAXIMIZE)
for i in M:    # naming the constraints r0,r1,r2,... (inside Gurobi)
    model.addConstr(sum(A[i,j]*x[j] + s[i] for j in N) == b[i], name='r'+str(i)) 

Discarded solution information


In [18]:
model.optimize()
if model.status != GRB.Status.OPTIMAL:
    print("***** Gurobi solve status:", model.status)
    print("***** This is a problem. Model does not have an optimal solution")
    raise StopExecution
print(" ")
print("primal var,      dual slack,      obj delta-lb,    obj delta-ub")
for j in N: print("x[",j,"]=",np.format_float_positional(x[j].X,4,pad_right=4), 
                  "  t[",j,"]=", np.format_float_positional(x[j].RC,4,pad_right=4),
                  "  L[",j,"]=", np.format_float_positional(x[j].SAObjLow-c[j],4,pad_right=4),
                  "  U[",j,"]=", np.format_float_positional(x[j].SAObjUp-c[j],4,pad_right=4))
print(" ")
print("dual vars,       primal slack,    rhs delta-lb,    rhs delta-ub")
for i in M: 
    constr=model.getConstrByName('r'+str(i))    # retriving from Gurobi the named constraints r0,r1,r2,...
    print("y[",i,"]=",np.format_float_positional(constr.Pi,4,pad_right=4), 
                  "  s[",i,"]=", np.format_float_positional(constr.Slack,4,pad_right=4),
                  "  L[",i,"]=", np.format_float_positional(constr.SARHSLow-b[i],4,pad_right=4),
                  "  U[",i,"]=", np.format_float_positional(constr.SARHSUp-b[i],4,pad_right=4))

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 5 PRO 5650U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 3 rows, 5 columns and 9 nonzeros
Model fingerprint: 0xba1a9232
Coefficient statistics:
  Matrix range     [2e+00, 8e+00]
  Objective range  [2e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 4e+01]
Presolve removed 0 rows and 3 columns
Presolve time: 0.00s
Presolved: 3 rows, 2 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2800000e+01   3.294000e+00   0.000000e+00      0s
       3    1.2125000e+01   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.212500000e+01
 
primal var,      dual slack,      obj delta-lb,    obj delta-ub
x[ 0 ]= 3.3750   t[ 0 ]= 0.       L[ 0 ]= -0.3333   U[ 0 ]= 0.2   
x[ 1 ]= 1. 

### We are now setting up the dual of this model $\min\{y'b ~:~ y'A\geq c',~ y\geq \mathbf{0}\}$.

In [37]:
model = gp.Model()
model.reset()
y = model.addMVar(m)
s = model.addMVar(n)
costobjective = model.setObjective(sum(y[i]*b[i] for i in M), GRB.MINIMIZE)
for j in N:    # naming the constraints r0,r1,r2,... (inside Gurobi)
    model.addConstr(sum(np.transpose(A[i,j])*y[i] - s[j] for i in M) == c[j], name='r'+str(j)) 

Discarded solution information


In [38]:
model.optimize()
if model.status != GRB.Status.OPTIMAL:
    print("***** Gurobi solve status:", model.status)
    print("***** This is a problem. Model does not have an optimal solution")
    raise StopExecution
print(" ")
print("primal var,      dual slack,      obj delta-lb,    obj delta-ub")
for i in M: print("y[",i,"]=",np.format_float_positional(y[i].X,4,pad_right=4), 
                  "  s[",i,"]=", np.format_float_positional(y[i].RC,4,pad_right=4),
                  "  L[",i,"]=", np.format_float_positional(y[i].SAObjLow-b[i],4,pad_right=4),
                  "  U[",i,"]=", np.format_float_positional(y[i].SAObjUp-b[i],4,pad_right=4))
print(" ")
print("dual vars,       primal slack,    rhs delta-lb,    rhs delta-ub")
for j in N: 
    constr=model.getConstrByName('r'+str(j))    # retriving from Gurobi the named constraints r0,r1,r2,...
    print("x[",j,"]=",np.format_float_positional(constr.Pi,4,pad_right=4), 
                  "  t[",j,"]=", np.format_float_positional(constr.Slack,4,pad_right=4),
                  "  L[",j,"]=", np.format_float_positional(constr.SARHSLow-c[j],4,pad_right=4),
                  "  U[",j,"]=", np.format_float_positional(constr.SARHSUp-c[j],4,pad_right=4))

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 5 PRO 5650U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 2 rows, 5 columns and 8 nonzeros
Model fingerprint: 0x62346732
Coefficient statistics:
  Matrix range     [3e+00, 8e+00]
  Objective range  [3e+01, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 3e+00]
Presolve removed 0 rows and 2 columns
Presolve time: 0.01s
Presolved: 2 rows, 3 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.250000e+00   0.000000e+00      0s
       2    1.2125000e+01   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.212500000e+01
 
primal var,      dual slack,      obj delta-lb,    obj delta-ub
y[ 0 ]= 0.25     s[ 0 ]= 0.       L[ 0 ]= -1.       U[ 0 ]= 1.    
y[ 1 ]= 0.1

# Optimal Values of both Primal and Dual match.